In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2025-10-01 15:31 ajna         INFO     Configuração de log efetuada


2025-10-01 15:31:33,596 ajna         INFO     Configuração de log efetuada


In [3]:
%load_ext autoreload 
%autoreload 2
engine = create_engine(SQL_URI)

# Recuperar informações de REDEX e OPERADORES

In [4]:
# Constantes organizando códigos
LISTA_OPERADORES = "('8931356', '8931359', '8931404', '8931318')"  # Terminais exportadores BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO
LISTA_REDEX = "('8931309', '8933204', '8931404')" # Redex da Localfrio/Movecta, Redex da Santos Brasil Clia e Redex da DPW/EMBRAPORT
FILTRO_LIKE_REDEX = "'89327%'"  # Abrange todos os outros Redex santistas

from datetime import datetime, timedelta

agora = datetime.now()
ha_X_dias = agora - timedelta(days=7)

# Parâmetros de data
data_final = agora.strftime('%Y-%m-%d %H:%M:%S')
data_inicial = ha_X_dias.strftime('%Y-%m-%d %H:%M:%S')

In [5]:
# SQL para entrada_operadores (alias e)
sql_entrada_operadores = f"""
SELECT 
    id, placa, placaSemirreboque, numeroConteiner, codigoRecinto, dataHoraOcorrencia, operacao, direcao, 
    cnpjTransportador, cpfMotorista, nomeMotorista, listaNfe, numeroConhecimento, vazioConteiner
FROM apirecintos_acessosveiculo
WHERE 
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'E'
    AND codigoRecinto IN {LISTA_OPERADORES}
    AND numeroConteiner !='' 
    AND vazioConteiner = 0 
"""

# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'E' é porque queremos a Entrada no Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)
# Este 'C' é porque é registro de acesso (entrada) e não agendamento
# Este '0' indica que o conteiner chegou estufado (cheio) ao Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)

In [6]:
# SQL para pesagem_operador - pesagem com recintos dos operadores (terminais exportadores)
sql_pesagem_operador = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_OPERADORES}
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

In [7]:
# SQL para pesagem_redex - pesagem com recintos Redex (estufadores)
sql_pesagem_redex = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

In [8]:
# SQL com JOIN para filtrar saidas do REDEX seguidas de entrada no Terminal Exportador (Operador portuário)
sql_saida_redex = f"""
SELECT DISTINCT
    c.id,
    c.placa,
    c.placaSemirreboque,
    c.numeroConteiner,
    c.vazioConteiner,
    c.codigoRecinto,
    c.cpfMotorista,
    c.nomeMotorista,
    c.dataHoraOcorrencia,
    c.direcao
FROM apirecintos_acessosveiculo AS c
JOIN apirecintos_acessosveiculo AS e
    ON e.numeroConteiner = c.numeroConteiner 
    AND e.placaSemirreboque = c.placaSemirreboque
    AND e.dataHoraOcorrencia >= c.dataHoraOcorrencia
    AND e.direcao = 'E'
    AND e.operacao = 'C'
    AND (
        (e.codigoRecinto NOT IN {LISTA_REDEX} AND e.codigoRecinto NOT LIKE {FILTRO_LIKE_REDEX})
        OR e.codigoRecinto = '8931404'
    )
    AND e.codigoRecinto != c.codigoRecinto
WHERE 
    c.dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND c.operacao = 'C'
    AND c.direcao = 'S'
    AND (c.codigoRecinto IN {LISTA_REDEX} OR c.codigoRecinto LIKE {FILTRO_LIKE_REDEX})
    AND c.numeroConteiner != ''
"""
# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'S' é porque queremos a Saida do Redex estufador
# Este 'C' é porque é registro de acesso (saída) e não agendamento
# A tabela c. é a tabela de ACESSO DE SAÍDA do REDEX no API RECINTOS
# A tabela e. é a tabela de ACESSO DE ENTRADA no TERMINAL EXPORTADOR (Operador Portuário)
# A lógica é que o caminhão com conteiner SAI DO REDEX ANTES DE ENTRAR NO TERMINAL EXPORTADOR (por isso o maior igual)
# Atenção! A DPW/EMBRAPORT pode operar como REDEX e envia carga para a BTP como pode receber carga de um REDEX e operar como Terminal exportador
# Este código contempla estes dois casos da DPW/EMBRAPORT

In [9]:
# Execução das consultas com parâmetros
df_entrada_operadores = pd.read_sql(
    text(sql_entrada_operadores), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [10]:
df_entrada_operadores

,id,placa,placaSemirreboque,numeroConteiner,codigoRecinto,dataHoraOcorrencia,operacao,direcao,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento,vazioConteiner
0,12660988,FEI3I38,FUY9846,SUDU6295757,8931318,2025-09-24 15:41:21,C,E,12014047000193,17898192885,ANTONIO CARLOS BRAGA,,,0
1,12660987,DPC9985,DPC9039,TGBU7994441,8931318,2025-09-24 15:41:25,C,E,32256956000145,04182055900,JOAO MARIANO GONCALVES,,,0
2,12660991,BXF6C32,SUU4A01,SEGU9440870,8931318,2025-09-24 15:52:16,C,E,01896217000194,50090005805,ARTHUR SOUZA LIRA,,152505289415224,0
3,12660995,BTT8F49,GAA2B04,MSNU8752410,8931318,2025-09-24 16:06:16,C,E,01896217000194,25601886865,EVERALDO DE SOUZA SANTOS,,,0
4,12660994,BUS4712,CUD4907,BEAU2148822,8931318,2025-09-24 16:06:40,C,E,12014047000193,03838154444,PEDRO ARAUJO DA SILVA,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22227,12782951,TIP6H86,TIT9A03,UETU2972298,8931404,2025-09-29 23:14:57,C,E,02916265009116,14592484819,MARCOS ANTONIO NASCIMENTO,35250902916265001131550010003694311201047057,,0
22228,12782953,ACB8110,BXE9H02,TXGU7756301,8931404,2025-09-29 23:15:22,C,E,21425093000176,34534704860,DIEGO BARROS DE ANDRADE,,,0
22229,12782957,FTC9I85,BYP7G60,HASU4801236,8931404,2025-09-29 23:18:02,C,E,02427026000146,46459831807,VINICIUS DE MORAES,35250900700460000122550010002338691913531475,,0
22230,12782958,MTR6E99,DPF3I53,EMCU3986671,8931404,2025-09-29 23:18:48,C,E,21425093000176,33962306803,TAMER BARBOSA,,,0


In [11]:
df_saida_redex = pd.read_sql(
    text(sql_saida_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [12]:
df_saida_redex

,id,placa,placaSemirreboque,numeroConteiner,vazioConteiner,codigoRecinto,cpfMotorista,nomeMotorista,dataHoraOcorrencia,direcao
0,12682669,DPB9906,CPJ4282,TCKU7709961,0,8932706,01850492816,JOAO LUIZ NIERO,2025-09-24 15:33:00,S
1,12684264,DTB5E78,NAU6H43,TTNU8307429,1,8932762,60941995534,JOSE RENILDO ALVES DA SILVA,2025-09-24 15:33:57,S
2,12683004,ESU7I75,CPJ4257,CMAU9493129,0,8932707,32680974808,DAVID MELO MESQUITA,2025-09-24 15:35:00,S
3,12708331,DVS2157,TJE3E31,MSMU1859851,1,8932797,13357678861,GERALDO DOMINGUES DE OLIVEIRA JUNIOR,2025-09-24 15:35:11,S
4,12684241,FJP5G01,TMG7A28,DFSU7022176,1,8932762,34532531837,JEFERSON ANDRE ROCHA,2025-09-24 15:36:05,S
...,...,...,...,...,...,...,...,...,...,...
6165,12812913,ABH3806,NFD9197,BEAU5431965,0,8932799,19943639830,CLAILTON SOUZA SANTOS,2025-09-30 22:41:51,S
6166,12812915,BTT6F50,DBM7A97,TRHU6068650,0,8932799,27122189813,MARCIO CARLOS DA SILVA,2025-09-30 22:43:27,S
6167,12812914,JYB8G42,ABQ3E61,TCLU6751012,0,8932799,06649059862,SEVERINO SILVIO FREIRE ALVES,2025-09-30 22:44:29,S
6168,12810695,BSG5C08,CPJ3D80,TCLU7896469,0,8932747,41576547817,AUGUSTO DOUGLAS DELIO,2025-09-30 22:51:30,S


In [13]:
df_saidas_entradas = pd.merge(df_entrada_operadores, df_saida_redex, how='inner', on=['placaSemirreboque', 'numeroConteiner'] , suffixes=('_E_Operador', '_S_REDEX'))
# Este 'inner' faz com que a tabela final obtida saidas_entradas só tenham dados que tem em Entrada Operador e Saída Rede
# Dados que não tenham ligação correspondente (são nulos) NÃO aparecerão
# Se você quiser que os não correspondentes também apareçam troque INNER por LEFT

In [14]:
df_saidas_entradas['transitTime'] = (df_saidas_entradas['dataHoraOcorrencia_E_Operador'] - 
                                     df_saidas_entradas['dataHoraOcorrencia_S_REDEX']).dt.total_seconds() / 3600


In [15]:
# df_saidas_entradas

In [16]:
#!pip install tqdm
from tqdm.notebook import tqdm

def buscar_id_fs_files(collection, numero_conteiner, data_ocorrencia):
    filtro = {
        'metadata.numeroinformado': numero_conteiner,
        'metadata.dataescaneamento': {
            '$gte': data_ocorrencia - timedelta(hours=3),
            '$lte': data_ocorrencia + timedelta(hours=6)
        }
    }
    resultado = collection.find_one(filtro, {'_id': 1})
    if resultado:
        return resultado['_id']
    return None

# Iterar sobre as linhas do DataFrame e aplicar busca
def mapear_ids_mongo(collection, df):
    ids_fs_files = []
    cont_sucesso = 0
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Buscando ids no MongoDB"):
        numero = row['numeroConteiner']
        data = row['dataHoraOcorrencia_E_Operador']
        id_fs = buscar_id_fs_files(collection, numero, data)
        ids_fs_files.append(id_fs)
        cont_sucesso += 1 if id_fs else 0
    df['fs_files_id'] = ids_fs_files
    print(f'Registros: {len(df)}, imagens: {cont_sucesso}')

In [17]:
mapear_ids_mongo(db['fs.files'], df_saidas_entradas)

Buscando ids no MongoDB:   0%|          | 0/5988 [00:00<?, ?it/s]

Registros: 5988, imagens: 2716


In [18]:
# df_saidas_entradas.transitTime.describe()
df_saidas_entradas.rename(columns={'placa': 'placa', 'placaSemirreboque': 'placa do semireboque', 'numeroConteiner': 'contêiner', 'vazioConteiner' : 'conteinervazio',
                         'codigoRecinto_S_REDEX': 'Redex', 'dataHoraOcorrencia_S_REDEX': 'Saída REDEX','placaSemirreboque': 'placa do semireboque',
                         'codigoRecinto_E_Operador': 'Operador', 'dataHoraOcorrencia_E_Operador': 'Entrada Operador',
                         'transitTime': 'transitTime', 'fs_files_id': '_id'}, inplace=True)
df_saidas_entradas.drop(['operacao', 'direcao_E_Operador', 'listaNfe', 'numeroConhecimento', 'direcao_S_REDEX'], axis=1)
df_saidas_entradas

,id_E_Operador,placa_E_Operador,placa do semireboque,contêiner,Operador,Entrada Operador,operacao,direcao_E_Operador,cnpjTransportador,cpfMotorista_E_Operador,...,id_S_REDEX,placa_S_REDEX,vazioConteiner_S_REDEX,Redex,cpfMotorista_S_REDEX,nomeMotorista_S_REDEX,Saída REDEX,direcao_S_REDEX,transitTime,_id
0,12661019,EIX7231,CUD4906,TCLU3127907,8931318,2025-09-24 16:58:47,C,E,12014047000193,31146520425,...,12676196,EIX7231,0,8931404,31146520425,ALCIDES F. DA SILVA FILHO,2025-09-24 15:56:05,S,1.045000,68d52215ea2419e0c125d680
1,12661045,JZN9882,FIM8788,HLBU1395928,8931318,2025-09-24 18:03:29,C,E,12014047000193,33355745839,...,12676333,JZN9882,0,8931404,33355745839,EDUARDO DE OLIVEIRA NASCIMENTO,2025-09-24 16:51:33,S,1.198889,68d52222e57654edfbeaf119
2,12661047,CUD3885,CUD3882,TRHU6725945,8931318,2025-09-24 18:09:47,C,E,12014047000193,03531935429,...,12676328,CUD3885,0,8931404,03531935429,EDERALDO MANOEL DE SOUSA,2025-09-24 16:42:41,S,1.451667,68d52222e57654edfbeaf12b
3,12661108,LPR3J16,CUD3880,KOCU2189796,8931318,2025-09-24 20:51:09,C,E,12014047000193,53415318095,...,12676538,LPR3J16,0,8931404,84860758404,OBERACIR GOMES DOS SANTOS,2025-09-24 19:51:23,S,0.996111,68d5223bea2419e0c125dd6e
4,12661131,EFW0757,FUZ7542,HMMU2237901,8931318,2025-09-24 22:09:30,C,E,12014047000193,21705880860,...,12676630,EFW0757,0,8931404,21705880860,MARCOS PINHEIRO FILHO,2025-09-24 20:34:10,S,1.588889,68d52247e57654edfbeaf48e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5983,12782766,MHR3I42,FSB1D39,TGBU9888840,8931404,2025-09-29 21:53:10,C,E,58188756000196,27315117881,...,12785409,MHR3I42,0,8932759,27315117881,ALEXANDRE MACKENZIE NOVAES,2025-09-29 20:58:31,S,0.910833,None
5984,12782865,RTN5I35,ECM3526,CICU3023536,8931404,2025-09-29 22:41:37,C,E,07275520000156,27919728894,...,12787559,RTN5I35,0,8932793,27919728894,MARCIO ANDRE BATISTA,2025-09-29 21:52:13,S,0.823333,68dbc05c2a1bc558aa62c2d7
5985,12782871,RNV7A47,GVQ8365,XHCU5486246,8931404,2025-09-29 22:44:52,C,E,07275520000156,16951257892,...,12787560,RNV7A47,0,8932793,16951257892,LUIZ CARLOS SANTOS LEITE,2025-09-29 21:54:13,S,0.844167,68dbc05c2a1bc558aa62c31b
5986,12782875,FQB2345,ECW1D57,LTRU4208050,8931404,2025-09-29 22:46:37,C,E,07275520000156,96933054934,...,12787562,FQB2345,0,8932793,96933054934,SIDNEI APARECIDO TIAGO,2025-09-29 22:00:27,S,0.769444,68dbc05c2a1bc558aa62c2fd


In [19]:
from IPython.display import display, HTML

def get_outliers(df):
    mean = df['transitTime'].mean()
    std = df['transitTime'].std()
    outliers = df[(df['transitTime'] < mean - 1 * std) | (df['transitTime'] > mean + 1 * std)]
    return outliers

def generate_html(_id):
    url = f"https://ajna1.rfoc.srf/virasana/file?_id={_id}"
    img = f"https://ajna1.rfoc.srf/virasana/imagens_cmap?_id={_id}"
    return f'<a href="{url}"><img src="{img}" width="200"></a>'
    
outliers = get_outliers(df_saidas_entradas)[['placa do semireboque', 'contêiner', 'Redex', 'Saída REDEX',
                                  'Operador', 'Entrada Operador', 'transitTime', '_id']]

outliers['html'] = outliers['_id'].apply(generate_html)
display(HTML(outliers.to_html(escape=False)))

,placa do semireboque,contêiner,Redex,Saída REDEX,Operador,Entrada Operador,transitTime,_id,html
1271,EPU9757,HLBU1594690,8932796,2025-09-24 16:28:07,8931356,2025-09-25 21:36:58,29.147500,None,
1308,DBM6886,HAMU1163326,8932796,2025-09-24 16:40:35,8931356,2025-09-26 00:21:26,31.680833,None,
1309,DBM6886,HAMU1163326,8932796,2025-09-24 16:59:05,8931356,2025-09-26 00:21:26,31.372500,None,
1310,DBM6886,HAMU1163326,8932796,2025-09-24 20:43:40,8931356,2025-09-26 00:21:26,27.629444,None,
1320,BRQ6G50,FANU3774265,8931404,2025-09-24 19:29:11,8931356,2025-09-26 01:13:21,29.736111,None,
1337,SWG6G68,HAMU2895194,8931404,2025-09-25 01:03:08,8931356,2025-09-26 02:02:30,24.989444,None,
1501,CZB2D55,HLBU1278650,8932759,2025-09-25 19:52:00,8931356,2025-09-26 10:19:20,14.455556,None,
1544,GJQ0I94,HAMU3643330,8932754,2025-09-24 20:24:30,8931356,2025-09-26 11:49:34,39.417778,None,
1743,TJK8J03,HLBU1983074,8931404,2025-09-25 01:00:50,8931356,2025-09-27 01:13:51,48.216944,None,
1916,DBB8E58,TRHU2533311,8931404,2025-09-26 22:02:55,8931356,2025-09-27 11:31:47,13.481111,68da76a1ea2419e0c1261f49,


In [20]:
# Adicionar células clicáveis que levem ao Ajna
df_saidas_entradas['contêiner'] = df_saidas_entradas['contêiner'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/bhadrasana2/consulta_container?numerolote={x}", "{x}")'
)
df_saidas_entradas['_id'] = df_saidas_entradas['_id'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/virasana/file?_id={x}", "{x}")'
)

# df_saidas_entradas

In [21]:
df_saidas_entradas.to_excel('saidas_entradas.xlsx', index=False)

In [22]:
df_pesagem_operador = pd.read_sql(
    text(sql_pesagem_operador), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

df_pesagem_redex = pd.read_sql(
    text(sql_pesagem_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)



# Seguem os merges e filtros conforme a lógica desejada especificada anteriormente
# Por exemplo, aqui unimos pesagem_redex com saida_redex e depois com entrada_operadores conforme passo a passo
df_pesagem_saida = pd.merge(df_pesagem_redex, df_saida_redex, on=['placaSemirreboque', 'codigoRecinto', 'numeroConteiner'], suffixes=('_pesagem', '_saida'))

df_pesagem_saida = df_pesagem_saida[
    (df_pesagem_saida['dataHoraOcorrencia_pesagem'] <= df_pesagem_saida['dataHoraOcorrencia_saida']) &
    ((df_pesagem_saida['dataHoraOcorrencia_saida'] - df_pesagem_saida['dataHoraOcorrencia_pesagem']).dt.days <= 7) &
    (df_pesagem_saida['direcao'] == 'S')
]

df_final = pd.merge(df_pesagem_saida, df_entrada_operadores, left_on=['placaSemirreboque', 'numeroConteiner'], right_on=['placaSemirreboque', 'numeroConteiner'], suffixes=('', '_entrada'))

df_final = df_final[
    ((df_final['dataHoraOcorrencia'] - df_final['dataHoraOcorrencia_saida']).dt.days <= 7) &
    (df_final['codigoRecinto'] != df_final['codigoRecinto_entrada']) 
]

df_final = df_final.drop_duplicates()

# Seleção de colunas para output
output_cols = [
    'dataHoraOcorrencia_pesagem', 'id_pesagem', 'placa', 'placaSemirreboque', 'pesoBrutoBalanca', 'numeroConteiner', 'codigoRecinto_pesagem',
    'dataHoraOcorrencia_saida', 'placa', 'codigoRecinto_saida', 'direcao', 'numeroConteiner',
    'dataHoraOcorrencia', 'placa', 'placaSemirreboque', 'id', 'cnpjTransportador', 'cpfMotorista', 'nomeMotorista', 'codigoRecinto', 'operacao', 'direcao', 'numeroConteiner', 'listaNfe', 'numeroConhecimento'
]

df_result = df_final[output_cols]

print(df_result.head())

KeyError: "['codigoRecinto_pesagem', 'codigoRecinto_saida'] not in index"